In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Perguntas e respostas com modelos generativos na Vertex AI

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Execute no Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      Veja no GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Execute no Vertex AI Workbench
    </a>
  </td>
</table>

## Visão geral

Grandes modelos de linguagem podem ser usados para várias tarefas de processamento de linguagem natural, incluindo perguntas e respostas (Q&A). Esses modelos são treinados em uma grande quantidade de dados de texto e podem gerar respostas de alta qualidade para uma ampla gama de perguntas. Uma coisa a observar aqui é que a maioria dos modelos tem datas limite em relação ao seu conhecimento, e perguntar qualquer coisa muito recente pode resultar em uma resposta incompleta, imaginativa ou incorreta (ou seja, uma alucinação).

Este bloco de anotações aborda os fundamentos dos prompts para responder a perguntas usando um modelo generativo. Além disso, apresenta o `domínio aberto` (conhecimento disponível na internet pública) e o `domínio fechado` (conhecimento que é mais privado - normalmente conhecimento empresarial ou pessoal).

Saiba mais sobre o design de prompt na [documentação oficial](https://cloud.google.com/vertex-ai/docs/generative-ai/text/text-overview#prompt_structure).

### Objetivo

Ao final deste notebook, você será capaz de escrever prompts para os seguintes cenários:

* Perguntas de **domínio aberto**:
     * Solicitação one-shot
     * Solicitação few-shot


* Perguntas de **domínio fechado**:
     * Fornecer conhecimento personalizado como contexto
     * Instrução de ajuste das saídas
     * Solicitação few-shot

### Custos
Este tutorial usa os seguintes componentes de Google Cloud:

* Vertex AI Generative AI Studio

Saiba mais sobre possíveis custos envolvidos [preços da Vertex AI](https://cloud.google.com/vertex-ai/pricing),
e use a [Calculadora de preços](https://cloud.google.com/products/calculator/)
para gerar uma estimativa de custo com base no uso projetado.

## Primeiros Passos

### Instalando os SDK da Vertex AI

In [ ]:
!pip install google-cloud-aiplatform --upgrade --user

**Somente Colab:** Descomente a célula a seguir para reiniciar o kernel ou use o botão para reiniciar o kernel.

In [ ]:
# # Reinicia automaticamente o kernel após as instalações para que seu ambiente possa acessar os novos pacotes
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### Autenticando seu ambiente de notebook
* Se você estiver usando o **Colab** para executar este notebook, descomente a célula abaixo e continue.
* Se você estiver usando o **Vertex AI Workbench**, confira as instruções de configuração [aqui](../setup-env/README.md).

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

### Importando as bibliotecas necessárias

**Somente Colab:** Descomente a célula a seguir para realizar o processo adequado de inicialização da SDK da Vertex AI.  

In [ ]:
# import vertexai

# PROJECT_ID = "[seu-project-id]"  # @param {type:"string"}
# vertexai.init(project=PROJECT_ID, location="us-central1")

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import warnings
warnings.simplefilter("ignore", UserWarning)

import pandas as pd
from vertexai.language_models import TextGenerationModel

#### Carregando o modelo `text-bison`

In [ ]:
generation_model = TextGenerationModel.from_pretrained("text-bison@001")

## Perguntas e respostas

Os recursos de resposta a perguntas exigem o fornecimento de um prompt ou uma pergunta que o modelo possa usar para gerar uma resposta. O prompt pode ser algumas palavras ou algumas frases completas, dependendo da complexidade da pergunta.

Ao criar um prompt de resposta a perguntas, é essencial ser específico e fornecer o máximo de contexto possível. Isso ajuda o modelo a entender a intenção por trás da pergunta e gerar uma resposta relevante. Por exemplo, se você quiser perguntar:

```
"Qual é a capital da França?",

então um bom prompt poderia ser:

"Diga-me o nome da cidade que é capital da França."

```

Além de ser específico, o prompt também deve ser gramaticalmente correto e livre de erros ortográficos. Isso ajuda o modelo a gerar uma resposta fácil de entender e contém menos erros ou imprecisões.

Ao fornecer prompts específicos e ricos em contexto, você pode ajudar o modelo a entender a intenção por trás da pergunta e gerar respostas precisas e relevantes.

Abaixo estão algumas diferenças entre as categorias **domínio aberto** e **domínio fechado** para prompts de resposta a perguntas.

* **Domínio aberto**: Todas as perguntas cujas respostas já estão disponíveis online. Eles podem pertencer a qualquer categoria, como história, geografia, países, política, química, etc. Isso inclui perguntas triviais ou de conhecimento geral, como:

```
P: Quem ganhou o ouro olímpico na natação?
P: Quem é o presidente de [dado país]?
P: Quem escreveu [livro específico]"?
```

Lembre-se do corte de treinamento de modelos generativos, pois perguntas envolvendo informações mais recentes do que aquelas com as quais o modelo foi treinado podem fornecer respostas incorretas ou imaginativas.


* **Domínio fechado**: Se você possui alguma base de conhecimento interna não disponível na internet pública, então ela pertence à categoria _domínio fechado_.
Você pode passar esse conhecimento "privado" como contexto para o modelo. Se solicitado corretamente, é mais provável que o modelo responda dentro do contexto fornecido e menos provável que dê respostas além da Internet aberta.

Considere o exemplo de criar um bot de perguntas e respostas sobre a documentação interna do produto. Nesse caso, você pode passar a documentação completa para o modelo e solicitar que ele responda apenas com base nisso.

Prompt típico para **domínio fechado**:

```
Prompt: f""" Resposta do contexto abaixo: \n\n
contexto: {sua base de conhecimento} \n
pergunta: {pergunta específica para essa base de conhecimento} \n
resposta: {a ser previsto pelo modelo} \n
"""
```

Abaixo estão alguns exemplos para entender esses diferentes tipos de prompts.

### Domínio aberto

#### Prompt zero-shot

In [ ]:
prompt = """Q: Quem era o presidente do Brasil em 2010?\n
            A:
         """
print(
    generation_model.predict(prompt, max_output_tokens=256, temperature=0.1).text
)

In [ ]:
prompt = """Q: Qual a montanha mais alta do mundo?\n
            A:
         """
print(
    generation_model.predict(prompt, max_output_tokens=20, temperature=0.1).text
)

#### Prompt few-shot

Digamos que você queira obter uma resposta curta do modelo (como apenas um nome específico). Para fazer isso, você pode aproveitar um prompt few-shot e fornecer exemplos ao modelo para ilustrar o comportamento esperado.

In [ ]:
prompt = """Q: Quem é atualmente o presidente da França?\n
         A: Emmanuel Macron \n\n

         Q: Quem inventou o telefone? \n
         A: Alexander Graham Bell \n\n

         Q: Quem escreveu o livro "1984"?
         A: George Orwell

         Q: Quem descobriu a penicilina?
         A:
         """
print(
    generation_model.predict(prompt, max_output_tokens=20, temperature=0.1).text
)

#### Prompt zero-shot vs few-shot

O prompt zero-shot pode ser útil para gerar texto rapidamente para novas tarefas, mas a qualidade do texto gerado pode ser inferior à de um prompt few-shot com exemplos bem escolhidos. O prompt few-shot geralmente é mais adequado para tarefas que exigem um alto grau de especificidade ou conhecimento específico do domínio, mas requer algum pensamento adicional e potencialmente dados para configurar o prompt.

### Domínio fechado

#### Adicionando conhecimento como contexto nos prompts

Imagine um cenário em que você gostaria de criar um bot de perguntas e respostas que aceita a documentação interna e permite que os usuários façam perguntas sobre ela.

No exemplo abaixo, o Google Cloud Storage e a documentação da política de conteúdo são adicionados ao prompt, para que a API PaLM possa usá-lo para responder a perguntas subsequentes com o contexto fornecido.

In [ ]:
context = """
Política de armazenamento e conteúdo \n
Qual é a durabilidade dos meus dados no Cloud Storage? \n
O armazenamento em nuvem foi projetado para durabilidade anual de 99,999999999% (11 9), o que é apropriado até mesmo para armazenamento primário e
aplicativos críticos para os negócios. Esse alto nível de durabilidade é alcançado por meio de codificação de eliminação que armazena partes de dados de forma redundante
em vários dispositivos localizados em várias zonas de disponibilidade.
Os objetos gravados no Cloud Storage devem ser armazenados de forma redundante em pelo menos duas zonas de disponibilidade diferentes antes do
a gravação é reconhecida como bem-sucedida. As somas de verificação são armazenadas e revalidadas regularmente para verificar proativamente se os dados
integridade de todos os dados em repouso, bem como para detectar corrupção de dados em trânsito. Se necessário, as correções são automaticamente
feito usando dados redundantes. Os clientes podem, opcionalmente, habilitar o controle de versão do objeto para adicionar proteção contra exclusão acidental.
"""

question = "Como podemos alcançar alta disponibilidade?"

prompt = f"""Responda a questão abaixo dada o contexto abaixo:
Contexto: {context} \n
Questão: {question} \n
Resposta:
"""

print(
    generation_model.predict(prompt).text
)

#### Ajustando as instruções de output do modelo

Outra maneira de ajudar os modelos de linguagem é fornecer instruções adicionais para enquadrar a saída no prompt. Para garantir que o modelo não responda a nada fora do contexto, o prompt pode especificar que a resposta deve ser "Informações não disponíveis no contexto fornecido", se for o caso.

In [ ]:
question = "Qual é a vantagem de habilitar o controle de versão no Cloud Storage?"
prompt = f"""Responda a pergunta abaixo utilizando as informações disponíveis como {{Context:}}. \n
Se a resposta não estiver disponível no {{Context:}} e você não esteja confiante no output, por favor
diga "Informação não disponível no contexto disponibilizado". \n\n"""
prompt += f"Contexto: {context} \n"
prompt += f"Pergunta: {question} \n"
prompt += "Resposta: "
print(f"[Pergunta]\n{question}")
print("[Resposta]")
print(
    generation_model.predict(prompt, max_output_tokens=256, temperature=0.3).text
)

print()

question = "Qual tipo de máquinas são requeridas para fazer o hosting de modelos na Vertex AI?"
prompt = f"""Responda a pergunta abaixo utilizando as informações disponíveis como {{Context:}}. \n
Se a resposta não estiver disponível no {{Context:}} e você não esteja confiante no output, por favor
diga "Informação não disponível no contexto disponibilizado". \n\n"""
prompt += f"Contexto: {context} \n"
prompt += f"Pergunta: {question} \n"
prompt += "Resposta: "
print(f"[Pergunta]\n{question}")
print("[Resposta]")
print(
    generation_model.predict(prompt, max_output_tokens=256, temperature=0.3).text
)

#### Prompt few-shot

In [ ]:
prompt = """
Contexto:
O termo "inteligência artificial" foi cunhado pela primeira vez por John McCarthy em 1956. Desde então, a IA se desenvolveu em um vasto
campo com inúmeras aplicações, desde carros autônomos até assistentes virtuais como Siri e Alexa.

Pergunta:
O que é inteligência artificial?

Responder:
A inteligência artificial refere-se à simulação da inteligência humana em máquinas programadas para pensar e aprender como humanos.

---

Contexto:
Os irmãos Wright, Orville e Wilbur, foram dois pioneiros da aviação americana a quem se atribui a invenção e
construindo o primeiro avião bem-sucedido do mundo e fazendo o primeiro voo humano controlado, motorizado e sustentado mais pesado que o ar,
  em 17 de dezembro de 1903.

Pergunta:
Quem eram os irmãos Wright?

Responder:
Os irmãos Wright foram pioneiros da aviação americana que inventaram e construíram o primeiro avião de sucesso do mundo.
e fez o primeiro voo humano controlado, motorizado e sustentado mais pesado que o ar, em 17 de dezembro de 1903.

---

Contexto:
A Mona Lisa é um retrato do século XVI pintado por Leonardo da Vinci durante o Renascimento italiano. é um dos
as pinturas mais famosas do mundo, conhecidas pelo sorriso enigmático da mulher retratada na pintura.

Pergunta:
Quem pintou a Mona Lisa?

Responder:
"""
print(generation_model.predict(prompt,).text)

### Perguntas e resposta extrativas

No próximo exemplo, o modelo generativo é guiado para entender o significado da pergunta e da passagem e identificar as informações relevantes na passagem que responde à pergunta. O modelo recebe uma pergunta e uma passagem de texto e é solicitado a encontrar a resposta para a pergunta dentro da passagem. A resposta é tipicamente uma frase ou sentença.

In [ ]:
prompt = """
Background: Há evidências de que houve mudanças significativas na vegetação da floresta amazônica ao longo dos últimos 21.000 anos através do Último Máximo Glacial (LGM) e subsequente deglaciação.
Análises de depósitos de sedimentos de paleolagos da bacia amazônica e do leque amazônico indicam que a precipitação na bacia durante o LGM foi menor do que no presente, e isso quase certamente foi
associada à reduzida cobertura de vegetação tropical úmida na bacia. Há um debate, no entanto, sobre quão extensa foi essa redução. Alguns cientistas argumentam que a floresta tropical foi reduzida a pequenas
refúgios isolados separados por floresta aberta e pastagens; outros cientistas argumentam que a floresta tropical permaneceu praticamente intacta, mas estendeu-se menos ao norte, sul e leste do que é visto hoje.
Este debate tem se mostrado difícil de resolver porque as limitações práticas de trabalhar na floresta tropical significam que a amostragem de dados é desviada do centro da bacia amazônica, e ambos
explicações são razoavelmente bem suportadas pelos dados disponíveis.

P: O que significa LGM?
R: Último Máximo Glacial.

P: O que indica a análise dos depósitos de sedimentos?
R: A precipitação na bacia durante o LGM foi menor do que no presente.

P: Quais são alguns dos argumentos dos cientistas?
R: A floresta tropical foi reduzida a pequenos refúgios isolados, separados por floresta aberta e pastagens.

P: Houve grandes mudanças na vegetação da floresta amazônica nos últimos quantos anos?
R: 21.000.

P: O que causou mudanças na vegetação da floresta amazônica?
R: O Último Máximo Glacial (LGM) e subsequente deglaciação

P: O que foi analisado para comparar as chuvas da Amazônia no passado e no presente?
R: Depósitos de sedimentos.

P: A que foi atribuída a menor precipitação na Amazônia durante o LGM?
R:
"""

print(generation_model.predict(prompt).text)

### Avaliação de respostas

Você pode avaliar os resultados da tarefa de pergunta e resposta se as respostas de verdade de cada pergunta estiverem disponíveis. No prompt zero-shot, você só pode usar perguntas de 'domínio aberto'. No entanto, com perguntas de 'domínio fechado', você pode adicionar contexto e avaliar de forma semelhante. Para mostrar como isso funciona, comece criando um dataframe simples com perguntas e respostas de verdade.

In [ ]:
qa_data = {
    "perguntas": [
        "Na barra de endereços dos navegadores, o que significa “www”?",
        "Quem foi a primeira mulher a ganhar um prêmio Nobel",
        "Qual é o nome do maior oceano da Terra?",
    ],
    "respostas_groundtruth": ["World Wide Web", "Marie Curie", "O oceano pacífico"],
}
qa_data_df = pd.DataFrame(qa_data)
qa_data_df

Agora que você tem os dados com perguntas e respostas básicas, pode chamar o modelo de geração PaLM 2 para cada linha de revisão usando a função `apply`. Cada linha usará o prompt dinâmico para prever a resposta usando a API PaLM. Vamos salvar os resultados na coluna `answer_prediction`.

In [ ]:
def geraResposta(row):
    prompt = f"""Responda as perguntas abaixo da forma mais precisa possível.\n\n
                 pergunta: {row}
                 resposta:
              """
    return generation_model.predict(prompt=prompt).text

qa_data_df["respostas_geradas"] = qa_data_df["perguntas"].apply(geraResposta)
qa_data_df

Você pode querer avaliar as respostas previstas pela API do PaLM. No entanto, será mais complexo do que a classificação do texto, pois as respostas podem diferir da verdade e podem ser apresentadas em um pouco mais/menos palavras.

Por exemplo, você pode observar a pergunta "Qual é o nome do maior oceano da Terra?" e veja que o modelo previu "Oceano Pacífico" quando um rótulo de verdade é "O Oceano Pacífico" com o extra "O". Agora, se você usar as métricas de classificação simples, considerará isso uma previsão errada, pois as strings originais e previstas têm uma diferença. No entanto, você pode ver que a resposta está correta, pois um "The" extra está causando o problema. É um problema simples de comparação de strings.

A solução para a comparação de strings em que `ground_thruth` e `predicted` podem ter algumas letras extras ou menos, uma abordagem é usar um algoritmo de correspondência difusa.
A correspondência de string difusa usa [Distância Levenshtein](https://en.wikipedia.org/wiki/Levenshtein_distance) para calcular as diferenças entre duas strings.

Aqui um exemplo usando a biblioteca `fuzzywuzzy`, que nos dá a `distância de Levenshtein` entre duas strings, mas em proporção. A pontuação bruta de proporção mede a similaridade da string como um int no intervalo [0, 100]. Para duas strings X e Y, a pontuação é definida por `int(round((2.0 * M / T) * 100))` onde `T` é o número total de caracteres em ambas as strings e `M` é o número de correspondências nas duas strings.

Leia mais aqui sobre a [fórmula de proporção](https://anhaidgroup.github.io/py_stringmatching/v0.3.x/Ratio.html):

Você pode ver um exemplo para entender melhor.
```
String1: "isto é um teste"
String2: "isto é um teste!"

Razão Fuzz => 97 #

Fuzz Partial Ratio => 100 #Como a maioria dos caracteres são iguais e em uma sequência semelhante, o algoritmo calcula a proporção parcial como 100 e ignora adições simples (novos caracteres).
```

Primeiro instale os pacotes `fuzzywuzzy` e `python-Levenshtein`:

In [ ]:
!pip install -q python-Levenshtein --upgrade --user
!pip install -q fuzzywuzzy --upgrade --user

E então calcule o match fuzzy:

In [ ]:
from fuzzywuzzy import fuzz


def get_fuzzy_match(df):
    return fuzz.partial_ratio(df["respostas_groundtruth"], df["respostas_geradas"])


qa_data_df["match_score"] = qa_data_df.apply(get_fuzzy_match, axis=1)
qa_data_df

Agora que você tem a pontuação de correspondência individual (parcial), pode obter a média ou a média de toda a coluna para ter uma noção dos dados gerais.

Pontuações próximas a 100 significam que o PaLM 2 pode prever com mais precisão; se a pontuação estiver próxima de 50 ou 0, não teve um bom desempenho.

In [ ]:
print(
    "Na média, o score de todas as predições do PaLM 2 foi: ",
    qa_data_df["match_score"].mean(),
    " %",
)

Nesse caso, você obteve um score de mais ou menos 97% como pontuação média, pois algumas previsões faltem algumas palavras. Isso significa que você está muito próximo da verdade básica e algumas respostas estão apenas perdendo a escrita exata da verdade básica.